In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import numpy as np
from datetime import datetime
import yfinance as yf
import os
import glob
import regex as re
import csv

Sentiment Analysis

In [20]:
%pip install transformers==4.25.1

  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.0.dev0
    Uninstalling transformers-4.26.0.dev0:
      Successfully uninstalled transformers-4.26.0.dev0
Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install textstat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [23]:
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
import numpy as np
import textstat

In [4]:
transformers.__version__

'4.25.1'

using finbert (91% accuracy) instead of Loughran and McDonalds (61% accuracy)
https://towardsdatascience.com/how-nlp-has-evolved-for-financial-sentiment-analysis-fb2990d9b3ed

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

sentiment_finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
sentiment_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

Clean Text First

In [11]:
mystr = """
I mean, I can comment a little bit about it. I mean, the corridor that we did very well in with Cuba and there is a â I don't know how else to explain it, but there's a black market currency and a regular currency. And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation.  And it's â and again, it's not just for us, it's for all of our competitors as well. They are all seeing the same deterioration.
"""
mystr

"\nI mean, I can comment a little bit about it. I mean, the corridor that we did very well in with Cuba and there is a â\x80\x93 I don't know how else to explain it, but there's a black market currency and a regular currency. And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation.  And it's â\x80\x93 and again, it's not just for us, it's for all of our competitors as well. They are all seeing the same deterioration.\n"

In [12]:
mystr2 = re.sub(r'[^A-Za-z0-9.:,!\' ]', '', mystr) #removes special characters such as the â from above
mystr2

"I mean, I can comment a little bit about it. I mean, the corridor that we did very well in with Cuba and there is a  I don't know how else to explain it, but there's a black market currency and a regular currency. And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation.  And it's  and again, it's not just for us, it's for all of our competitors as well. They are all seeing the same deterioration."

In [13]:
mystr3 = re.sub('\s+', ' ', mystr2) #removes multiple spaces to a single space (i.e. there is a  I --> there is a I)
mystr3

"I mean, I can comment a little bit about it. I mean, the corridor that we did very well in with Cuba and there is a I don't know how else to explain it, but there's a black market currency and a regular currency. And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation. And it's and again, it's not just for us, it's for all of our competitors as well. They are all seeing the same deterioration."

In [14]:
sentiment_nlp = pipeline("text-classification", model=sentiment_finbert, tokenizer=sentiment_tokenizer)
results = sentiment_nlp(mystr3)
results

[{'label': 'Neutral', 'score': 0.9691150188446045}]

tokenise string to individual sentences:

In [4]:
import nltk.data

nltk_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [137]:
mystrtok = (nltk_tokenizer.tokenize(mystr3))
mystrtok

['I mean, I can comment a little bit about it.',
 "I mean, the corridor that we did very well in with Cuba and there is a I don't know how else to explain it, but there's a black market currency and a regular currency.",
 "And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation.",
 "And it's and again, it's not just for us, it's for all of our competitors as well.",
 'They are all seeing the same deterioration.']

In [ ]:
# FLS-Classification:

# Forward-looking statements (FLS) inform investors of managers’ beliefs and opinions about firm's future events or results. Identifying forward-looking statements from corporate reports can assist investors in financial analysis. FinBERT-FLS is a FinBERT model fine-tuned on 3,500 manually annotated sentences from Management Discussion and Analysis section of annual reports of Russell 3000 firms.

# There are 3 classifications of FLS:
# Specific foward looking statment  (S-FLS)
# Non-specific foward looking statment (NS-FLS)
# Not foward looking statment (N-FLS)

# We are trying to take the average sentiments of all different classes of FLS statements.
# E.g.
# 4 sentences that are classed as S-FLS, with sentiments [0.6, 0.7, 0.8, 0.9]
# so the average sentiment is 0.75
# --> Hence, repeat for NS-FLS and NFLS

# But first, I need to change the sentiment into a single numerical value
# i.e. {'label': 'Negative', 'score': 0.5338950753211975} --> single value

# so for 
# sentence1 = {'label': 'Negative', 'score': 0.5338950753211975}
# sentence2 = {'label': 'Neutral', 'score': 0.999138355255127}
# sentence3 = {'label': 'Positive', 'score': 0.9999885559082031}

# negative can be mapped to -1
# neutral to 0
# positive to 1

# the sentiment score probability is multiplied by their -1, 0 or 1

# so:
# sentence1 = -0.5338950753211975
# sentence2 = 0
# sentence 3 = 0.9999885559082031

# ------------------------------------------------------------------------------------------------
# given 
# [{'label': 'Not FLS', 'score': 0.9779328107833862},
#  {'label': 'Not FLS', 'score': 0.9796538949012756},
#  {'label': 'Specific FLS', 'score': 0.8796855211257935}]

# and given
# [{'label': 'Negative', 'score': 0.5338950753211975},
#  {'label': 'Neutral', 'score': 0.999138355255127},
#  {'label': 'Positive', 'score': 0.9999885559082031}]

# Sentiment score of Not FlS = (-0.5339+0)/2 = -0.26695
# Sentiment score of Specific FLS = 0.99998

In [6]:
# FLS classification
fls_finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
fls_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')

In [5]:
nltkstr = "Marcelo Fischer : Just to correct the income from operations, the loss from operations for net2phone was minus $1.8 billion. Of EBITDA itself was about minus $100,000, okay? So we are getting closer to reaching EBITDA profitability and based on our 2023 projections, we do hope to exit 2023 with net2phone being an EBITDA-positive company, even as it continues to grow at rates which are probably higher than the average UCaaS play on basically."

nltkstr= (nltk_tokenizer.tokenize(nltkstr))
nltkstr

['Marcelo Fischer : Just to correct the income from operations, the loss from operations for net2phone was minus $1.8 billion.',
 'Of EBITDA itself was about minus $100,000, okay?',
 'So we are getting closer to reaching EBITDA profitability and based on our 2023 projections, we do hope to exit 2023 with net2phone being an EBITDA-positive company, even as it continues to grow at rates which are probably higher than the average UCaaS play on basically.']

In [7]:
#performs FLS-classifications on the above 3 sentences
fls_nlp = pipeline("text-classification", model=fls_finbert, tokenizer=fls_tokenizer)
fls_results = fls_nlp(nltkstr)
fls_results

[{'label': 'Not FLS', 'score': 0.9779328107833862},
 {'label': 'Not FLS', 'score': 0.9796538949012756},
 {'label': 'Specific FLS', 'score': 0.8796855211257935}]

In [31]:
sentiment_results = sentiment_nlp(nltkstr)
sentiment_results

[{'label': 'Negative', 'score': 0.5338950753211975},
 {'label': 'Neutral', 'score': 0.999138355255127},
 {'label': 'Positive', 'score': 0.9999885559082031}]

In [22]:
# Word Complexity:
# https://pypi.org/project/textstat/
# I chose Flesch Reading Ease and the Gunning Fog Index because
# The Flesch Reading Ease score is a simple and widely used readability formula that can quickly give you an idea of how easy or difficult the text is to read. It uses the average sentence length and the average number of syllables per word to calculate the score. A higher score indicates that the text is easier to read, while a lower score indicates that the text is more difficult to read. This could help you identify if the language used in the call is easily understood by the general public.
# The Gunning Fog Index, on the other hand, is a more sophisticated readability formula that is often used for business and government documents. It also uses the average sentence length and the average number of words per sentence to calculate the number of years of education needed to understand a text. This readability measure is more suited to evaluate the complexity of the texts, and indicate the level of sophistication of the call.

In [18]:
fls_results = fls_nlp(nltkstr)
print(fls_results)
n_flslist = []
s_flslist = []
ns_flslist = []
for i in range(0, len(nltkstr)):
    sentence = nltkstr[i]
    if fls_results[i]['label'] == 'Not FLS':
        n_flslist.append(sentence)
    elif fls_results[i]['label'] == 'Specific FLS':
        s_flslist.append(sentence)
    elif fls_results[i]['label'] == 'Non-specific FLS':
        ns_flslist.append(sentence)


[{'label': 'Not FLS', 'score': 0.9779328107833862}, {'label': 'Not FLS', 'score': 0.9796538949012756}, {'label': 'Specific FLS', 'score': 0.8796855211257935}]


In [19]:
# sentences that areclassed as "Not FLS"
n_flslist

['Marcelo Fischer : Just to correct the income from operations, the loss from operations for net2phone was minus $1.8 billion.',
 'Of EBITDA itself was about minus $100,000, okay?']

In [20]:
# sentences that areclassed as "Specific FLS"
s_flslist

['So we are getting closer to reaching EBITDA profitability and based on our 2023 projections, we do hope to exit 2023 with net2phone being an EBITDA-positive company, even as it continues to grow at rates which are probably higher than the average UCaaS play on basically.']

In [21]:
# sentences that areclassed as "Not Specific FLS"
ns_flslist

[]

In [52]:
n_flsliststr = ' '.join(n_flslist) # turns all of the non fls sentences into one paragraph for future analysis.

# maps sentiment data so it outputs a single sentiment value
def map_sentiments(sentiment_result):
    sentiment_result = sentiment_result[0]
    if sentiment_result['label'] == 'Negative':
        return -1 * sentiment_result['score']
    
    elif sentiment_result['label'] == 'Neutral':
        return 0
    
    elif sentiment_result['label'] == 'Positive':
        return sentiment_result['score']

def get_NLP_values(liststr):
    # further analysis includes finding sentiment and word complexity.
    if len(liststr) == 0:
        return 0, 0, 0
    
    else:
        # maps sentiment data so it outputs a single sentiment value
        sentiment_result = sentiment_nlp(liststr)
        # gets 
        sentiment_score = map_sentiments(sentiment_result)

        # word complexity:
        flesch_score = textstat.flesch_reading_ease(liststr)
        gunning_fog_score = textstat.gunning_fog(liststr)

        return sentiment_score, flesch_score, gunning_fog_score

In [53]:
n_flsliststr = ' '.join(n_flslist) # turns all of the non fls sentences into one paragraph for future analysis.
s_flsliststr = ' '.join(s_flslist)
ns_flsliststr = ' '.join(ns_flslist)

n_fls_sentiment_score, n_fls_flesch_score, n_fls_gunning_fog_score = get_NLP_values(n_flsliststr)
s_fls_sentiment_score, s_fls_flesch_score, s_fls_gunning_fog_score = get_NLP_values(s_flsliststr)
ns_fls_sentiment_score, ns_fls_flesch_score, ns_fls_gunning_fog_score = get_NLP_values(ns_flsliststr)


print(n_fls_sentiment_score, n_fls_flesch_score, n_fls_gunning_fog_score)
print(s_fls_sentiment_score, s_fls_flesch_score, s_fls_gunning_fog_score)
print(ns_fls_sentiment_score, ns_fls_flesch_score, ns_fls_gunning_fog_score)

# 0      -->    neutral sentiment
# 57.77  -->    flesch_score value
# 8.36   -->    gunning_fog_score value


0 57.77 8.36
0.9999885559082031 24.79 24.49
0 0 0


In [ ]:
# List of .csv file info for each stock:
# ------------------------------------------------------------------------------------------------
# META DATA
# ------------------------------------------------------------------------------------------------
#
# 0. Year of transcript release
# 1. Quarter of transcript release
# 2. Date of transcript release
# 3. Earnings Transcript contents
#
# ------------------------------------------------------------------------------------------------
# Feature Extractions:
# ------------------------------------------------------------------------------------------------
#
# 4. EPS surprise value
#
# ------------------------------------------------------------------------------------------------
# Transcript Features:
# ------------------------------------------------------------------------------------------------
#
# 5. Whole transcript - net sentiment
# 6. Whole transcript - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 7. Whole transcript - net word complex
#
# ------------------------------------------------------------------------------------------------
#
# Pre release:
# 8. Whole pre-release - net sentiment
# 9. Whole pre-release - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 10. Whole pre-release - net word complexity
#
#
# 11. Specific foward looking statment - sentiment
# 12. Specific foward looking statment - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 13. Specific foward looking statment - word complexity
#
# 14. Non Specific Forward looking statement - sentiment 
# 15. Non Specific Forward looking statement - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 16. Non Specific Forward looking statement - word complexity
#
# 17. Not Foward looking statement - sentiment
# 18. Not Foward looking statement - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 19. Not Foward looking statement - word complexity
#
# ------------------------------------------------------------------------------------------------
#
# Questions & Answers:
# 20. Whole Q&A - net sentiment
# 21. Whole Q&A – #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 22. Whole Q&A - net word complexity
# 
# 23. all question (aggregate) - sentiment
# 24. all question (aggregate) – #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 25. all question (aggregate) - word complex
# 26. all reply (aggregate) - sentiment
# 27. all reply (aggregate) – #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 28. all reply (aggregate) - word complex
#
# For all replies (aggregate):
# 29: Specific foward looking statment - sentiment 
# 30: Specific foward looking statment – #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 31: Specific foward looking statment - word complexity
# 32: Non Specific Forward looking statement - sentiment
# 33: Non Specific Forward looking statement – #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 34: Non Specific Forward looking statement - word complexity
# 35: Not Foward looking statement - sentiment
# 36: Not Foward looking statement - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 37: Not Foward looking statement - word complexity
#
# ------------------------------------------------------------------------------------------------
#
# With specific words analysis:
# Sentences that includes the word:
# all of these words can be plural (e.g. cost and costs)
# 38: "margin" - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 39: "cost" - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 40: "revenue" - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 41: "earnings" - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 42: "growth" - #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 43: "EBIDTA" -  #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 44: "leverage" -  #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 45: "debt" -  #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# 46: "price" – #of_positive(sentiment)/#of_negative+positive+neutral(sentiment)
# ------------------------------------------------------------------------------------------------
#
# 47: Rolling frame of Whole Transcripts - TF-IDF value
# 48: Rolling frame of Pre-releases - TF-IDF value
# 49: Rolling frame of Management Sentences (Their Replies) - TF-IDF
#
# ------------------------------------------------------------------------------------------------
#
# 50: Stock price difference between Day 0 (Day earnings call is released) and Day 10
# 51: Stock price difference between Day 0 (Day earnings call is released) and Day 20
# 52: Stock price difference between Day 0 (Day earnings call is released) and Day 30
# 53: Stock price difference between Day 0 (Day earnings call is released) and Day 40
# 54: Stock price difference between Day 0 (Day earnings call is released) and Day 50
# 55: Stock price difference between Day 0 (Day earnings call is released) and Day 60
# 56: Stock price difference between Day 0 (Day earnings call is released) and Day 70
# 57: Stock price difference between Day 0 (Day earnings call is released) and Day 80
# 58: Stock price difference between Day 0 (Day earnings call is released) and Day 90
#
# 59: Stock price difference between Day 1 (Day earnings call is released) and Day 10
# 60: Stock price difference between Day 1 (Day earnings call is released) and Day 20
# 61: Stock price difference between Day 1 (Day earnings call is released) and Day 30
# 62: Stock price difference between Day 1 (Day earnings call is released) and Day 40
# 63: Stock price difference between Day 1 (Day earnings call is released) and Day 50
# 64: Stock price difference between Day 1 (Day earnings call is released) and Day 60
# 65: Stock price difference between Day 1 (Day earnings call is released) and Day 70
# 66: Stock price difference between Day 1 (Day earnings call is released) and Day 80
# 67: Stock price difference between Day 1 (Day earnings call is released) and Day 90


In [ ]:
TF IDF value